# Data Exploration and Preprocessing Notebook

In [1]:
import json
import regex
from datetime import datetime, timezone
import pandas as pd

## 1- Extracting relevant data

The set of keys that we will consider in our sentiment analysis task are the following:
- `selftext`: the text of the post, will contain nuances of the sentiment
- `title`: the title of the post often refers to the asset or the topic of the post
- `created` or `created_utc`: is the date of creation of the post

We removed submissions that had `[deleted]` or `[removed]` in either their text or title. <br>
This means that the post was moderated and we cannot use it for our analysis. <br>

In [ ]:
# create a new json file with data that only contains the above keys
data = []
with open("Data/wallstreetbets_submissions", "r") as f:
    for line in f:
        d = json.loads(line)
        # check if the title or text contains the string "[removed]"
        if "[removed]" not in d['title'] and "[removed]" not in d['selftext'] and "[deleted]" not in d['title'] and "[deleted]" not in d['selftext']:
            data.append(
                {
                    'date': d['created_utc'] if 'created_utc' in d else d['created'],
                    'title': d['title'],
                    'text': d['selftext'],
                }
            )

There are **822004** such submissions that we will further preprocess in order to perform sentiment analysis.<br>
These submissions are going to aliment our sentiment analysis model.

In [ ]:
len(data)

The cleaned data file can be found under `Data/wallstreetbets_submissions_cleaned.json`

In [ ]:
# write the data to a new file in a well formatted way
with open("Data/wallstreetbets_submissions_cleaned.json", "w") as f:
    json.dump(data, f, indent=4)

## 2- SPY Data

Now we are going to check how many submissions are related to the S&P 500 ETF (SPY).<br>
For this we will be looking for the keywords:
- `SPY`
- `SP500`
- `S&P500`
- `S&P 500`
- `S&P`
- `Standard & Poor's`
- `Standard and Poor's`
- `Standard and Poor 500`
- `Standard & Poor 500`
- `Standard and Poor 500`
- `Standard & Poor`
- `Standard and Poor`
- `Standard and Poor's 500`
- `Standard & Poor's 500`
- `Standard and Poor's 500`
in the title and the text <br>
We will be using regexes to find these keywords in an optimized way.

In [ ]:
# create a list of regexes that match the words in the above list
words = ["spy", "sp500", "s&p", "standard & poor", "standard and poor"]

# create a regex pattern that matches any of the words in the above list
pattern = regex.compile(r'\b(?:' + '|'.join(words) + r')\b', regex.IGNORECASE)

In [ ]:
spy_data = []

with open("Data/wallstreetbets_submissions_cleaned.json", "r") as f:
    data = json.load(f)
    for d in data:
        if pattern.search(d['title']) or pattern.search(d['text']):
            spy_data.append(d)

The data related to SPY can be found under `Data/wallstreetbets_submissions_SPY.json`

In [ ]:
with open("Data/wallstreetbets_submissions_spy.json", "w") as f:
    json.dump(spy_data, f, indent=4)

To have an idea on the period that is spanned by the data related to SPY, we will plot the number of submissions per day.

In [ ]:
date_count = {}
utc_zone = timezone.utc

for d in spy_data:
    date = datetime.fromtimestamp(int(d['date']), tz=utc_zone)
    if date in date_count:
        date_count[date] += 1
    else:
        date_count[date] = 1

## 3- Finding the right assets

We are going to check the data for keywords that are related to specific stocks. <br>
We will be looking for the keywords:
- `AAPL`
- `TSLA`
- `NVDA`
- `AMZN`
- `LVMH`

In [ ]:
amzn = [
    "amazon",
    "primeday", 
    "alexa", 
    "kindle", 
    "aws",
    "amzn",
    "prime",
    "bezos",
    "e-commerce giant"
]

tsla = [
    "tsla",
    "tesla",
    "elon",
    "musk",
    "tesla",
    "model s",
    "model x",
    "model 3",
    "model y",
    "gigafactory",
    "electric vehicles",
    "ev",
    "autopilot"
]

nvda = [
    "nvda",
    "nvidia",
    "jensen huang",  # CEO of nvidia
    "geforce",  # nvidia's popular gPU brand
    "turing",  # nvidia's gPU architecture
    "ampere",  # A more recent nvidia gPU architecture
    "rtx",  # Ray tracing gPUs
    "gtx",  # Previous generation gPUs
    "quadro",  # Professional graphics cards
    "tesla gpu",  # not to be confused with Tesla Motors
    "cuda",  # nvidia's parallel computing platform
    "dlss",  # Deep Learning Super Sampling, nvidia's AI-driven image upscaling technology
    "ray tracing",  # graphics rendering technique that nvidia gPUs support
    "g-sync",  # nvidia's display technology
    "omniverse"  # nvidia's collaboration and simulation platform
]

aapl = [
    "aapl",
    "apple",
    "tim cook"
    "steve jobs"
    "iphone",
    "ipad",
    "mac",
    "ios",
    "app store",
    "icloud",
    "siri",
    "wwdc",  # Apple's Worldwide Developers Conference
    "m1",
    "m2",
    "face id",
    "touch id",
    "homepod",
    "airtag",
    "Aarkit",  # Apple's AR platform
    "carplay",
    "airpods"
]

lvmh = [
    "lvmh",
    "bernard arnault",
    "louis vuitton",
    "dior",
    "givenchy",
    "Fendi",
    "céline",
    "kenzo",
    "marc jacobs",
    "Hublot",
    "tag Heuer",
    "sephora",
    "moët & chandon",
    "dom pérignon",
    "hennessy",
    "cognac",
    "loro piana",
    "dfs group",
    "la samaritaine",
    "cheval blanc",
]

wmt = [
    "wmt",
    "walmart",
    "doug mcmillon",
    "sam walton",
    "walton family",
    "supercenter",
    "hypermarket",
    "neighborhood market",
    "sam's"
]

goog = [
    "goog",
    "google",
    "alphabet",
    "larry page",
    "sergey brin",
    "sundar pichai",
    "android",
    "chrome",
    "search",
    "maps",
    "youtube",
    "play store",
    "pixel",
    "nexus",
    "nest",
    "waymo",
    "deepmind",
    "loons",
    "sidewalk labs",
    "fitbit",
    "waze",
    "doubleclick",
    "admob",
    "adsense",
    "adwords",
    "gmail",
    "drive",
    "photos",
    "podcasts",
    "books",
    "stadia",
    "pay",
    "wallet",
    "shopping",
    "one",
    "workspace",
    "cloud",
    "gsuite",
    "firebase",
    "colab",
    "bigquery",
    "kubernetes",
    "chromeos",
    "chromebook",
    "chromecast",
    "nexus",
    "pixel",
    "pixelbook",
    "pixelbuds",
    "voice",
    "hangouts",
    "duo",
    "meet",
    "gmail",
    "maps",
    "street view"
]

spy = ["spy", "sp500", "s&p", "standard & poor", "standard and poor"]

In [ ]:
utc_zone = timezone.utc

def check_keywords(comments, keywords):
    """
    This function checks if any of the keywords are present in the title or text of the submission.
    It returns a dict of {date: count} where count is the number of keywords found in the submission.
    """
    # create a regex pattern that matches any of the words in the above list
    pattern = regex.compile(r'\b(?:' + '|'.join(keywords) + r')\b', regex.IGNORECASE)

    keyword_count = {}
    for c in comments:
        # look for the keywords in the title and text of the submission in lower case using the regex library
        if pattern.search(c['text'].lower()) or pattern.search(c['title'].lower()):
            date = datetime.fromtimestamp(int(c['date']), tz=utc_zone).date()
            if date in keyword_count:
                keyword_count[date] += 1
            else:
                keyword_count[date] = 1

    return keyword_count

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import numpy as np  # Needed for mathematical operations like mean and median

def plot_date_counts(title, date_counts):
    # Sort the dictionary by date to ensure the plot is ordered
    sorted_dates = sorted(date_counts.items())
    dates, counts = zip(*sorted_dates)

    # Calculate statistics
    mean_count = np.mean(counts)
    median_count = np.median(counts)
    max_count = max(counts)
    min_count = min(counts)

    # Create the plot
    plt.figure(figsize=(10, 5))
    plt.plot(dates, counts, marker='o', linestyle='-', color='b', markersize=2, linewidth=1)

    # Add horizontal lines for mean and median
    plt.axhline(y=mean_count, color='r', linestyle='--', linewidth=1, label=f'Mean: {mean_count:.2f}')
    plt.axhline(y=median_count, color='g', linestyle=':', linewidth=1, label=f'Median: {median_count:.2f}')

    # Formatting the date display on the x-axis
    plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator(minticks=3, maxticks=7))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    plt.gcf().autofmt_xdate()  # Rotation

    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend()  # Add legend to the plot
    plt.tight_layout()

    # Display statistics in the plot
    plt.figtext(0.99, 0.01, f'Max: {max_count}\nMin: {min_count}', horizontalalignment='right')

    plt.show()

In [ ]:
with open("Data/wallstreetbets_submissions_cleaned.json", "r") as f:
    comments = json.load(f)

### Checking TSLA:

In [ ]:
tsla_dict = check_keywords(comments, tsla)
nvda_dict = check_keywords(comments, nvda)
amzn_dict = check_keywords(comments, amzn)
aapl_dict = check_keywords(comments, aapl)
lvmh_dict = check_keywords(comments, lvmh)
wmt_dict = check_keywords(comments, wmt)
goog_dict = check_keywords(comments, goog)
spy_dict = check_keywords(comments, spy)

In [ ]:
# get the date of the maximum count for each stock
print("tsla max date: ", max(tsla_dict, key=tsla_dict.get))
print("nvda max date: ", max(nvda_dict, key=nvda_dict.get))
print("amzn max date: ", max(amzn_dict, key=amzn_dict.get))
print("aapl max date: ", max(aapl_dict, key=aapl_dict.get))
print("lvmh max date: ", max(lvmh_dict, key=lvmh_dict.get))
print("wmt max date: ", max(wmt_dict, key=wmt_dict.get))
print("goog max date: ", max(goog_dict, key=goog_dict.get))
print("spy max date: ", max(spy_dict, key=spy_dict.get))


In [ ]:
plot_date_counts("TSLA mentions", tsla_dict)

In [ ]:
plot_date_counts("NVDA mentions", nvda_dict)

In [ ]:
plot_date_counts("AMZN mentions", amzn_dict)

In [ ]:
plot_date_counts("AAPL mentions", aapl_dict)

In [ ]:
plot_date_counts("LVMH mentions", lvmh_dict)

In [ ]:
plot_date_counts("WMT mentions", wmt_dict)

In [ ]:
plot_date_counts("GOOG mentions", goog_dict)

In [ ]:
plot_date_counts("SPY mentions", spy_dict)

In [ ]:
df = pd.read_csv("Data/tweets_labelled_09042020_16072020.csv", sep=";")

In [ ]:
# count non nan sentiment values
sentiment_count = df['sentiment'].value_counts()
print(sentiment_count)

In [ ]:
df2 = pd.read_csv("Data/stock_data.csv", sep=",")

In [ ]:
sentiment_count = df2['Sentiment'].value_counts()
print(sentiment_count)

## 4- Creating the Dataset of sentiments

In [18]:
# load the spy data
with open("Data/wsb_spy.json", "r") as f:
    spy_data = json.load(f)

# create a dataframe from the spy data
spy_df = pd.DataFrame(spy_data)

spy_prices = pd.read_csv("Data/spy_prices.csv")

In [19]:
# convert the date column to datetime in month format
spy_df['date'] = pd.to_datetime(spy_df['date'], unit='s').dt.to_period('M')

/var/folders/zj/sr110gkn1vz4qf70llfwlh8c0000gn/T/ipykernel_3334/8274418.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  spy_df['date'] = pd.to_datetime(spy_df['date'], unit='s').dt.to_period('M')


In [20]:
# group the data by date and take the average sentiment and the count of submissions
spy_grouped = spy_df.groupby('date').agg({'prediction': 'mean', 'text': 'count'}).reset_index()

In [22]:
# change the column Date of the spy_prices dataframe to datetime format period month
spy_prices['Date'] = pd.to_datetime(spy_prices['Date']).dt.to_period('M')

# change the name of the column Date to date
spy_prices = spy_prices.rename(columns={'Date': 'date'})

In [23]:
# merge the two dataframes on the date column and only keep the column Close from the spy_prices dataframe
spy_grouped = spy_grouped.merge(spy_prices[['date', 'Close']], on='date')

In [26]:
# rename the columns to something more meaningful
spy_grouped = spy_grouped.rename(columns={'prediction': 'average_sentiment', 'text': 'submission_count', 'Close': 'close_price'})

In [27]:
# write spy_grouped to a csv file
spy_grouped.to_csv("Data/spy_data.csv", index=False)